# COGS 108 - Final Project

## Important

- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'FinalProject.ipynb'.
- Only upload the .ipynb file to TED, do not upload any associted data. Make sure that for cells in which you want graders to see output that these cells have been executed.

## Group Members: Fill in the Student IDs of each group member here

Replace the lines below to list each persons full student ID, ucsd email and full name.

- A12785571, ekfu@ucsd.edu, Edwin Fu
- A12623303, jcloo@ucsd.edu, Joel Loo
- A13589612, bfbarry@ucsd.edu, Brian Barry
- A12690911, zes004@ucsd.edu, Sunny Sun
- A13736482, roz021@ucsd.edu, Roy Zhang
- A13014470, khh007@ucsd.edu, Kevin Huang


## Introduction: Background

From 2013-2018, the United States of America has gone through a period of time when there were over 260,000 reported occurrence of gun violence. With so many reported incidents within the span of only 5 years, there needs to be a way for citizens to find where all these incidents are occurring without having to sift through heaps of data. People should be aware of where areas of high gun violence rates are occurring and how possible prevention can occur. We seek to find out whether or not gun violence rates, household income level, and population level are correlated and if so, can there be a way to lower the gun violence rates in those areas. To operationalize this, we are using gun violence rates, household income levels, and population counts as our metrics since all three can be utilized for our research question of whether there is a correlation between income and gun violence within an area of the United States.

To start with our analysis, we will start by importing some Python library packages for analyzing and graphing our data:


# Library imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

plotly.tools.set_credentials_file(username='jcloo', api_key='6Qa08K4DBMMNjPHL2p0T')


In [3]:
# Configure Seaborn library
sns.set()
sns.set_context('talk')

In [4]:
#read datasets
url = "https://doc-0k-88-docs.googleusercontent.com/docs/securesc/a4oc2hqjnhqph43vaufe9fs3nelfc15u/n2em47vd7bqh0bu7a279knuoq26ugn8a/1552176000000/12632076152020903193/12632076152020903193/1KXzWFA-RIXwBXuOu81-E-MYi-e7WVfTj?e=download&nonce=1r1nehh1hfp4m&user=12632076152020903193&hash=uc1qcip4tede3ao389g85jg856941gf0"
gun_data = pd.read_csv("datasets/gun-violence-data_01-2013_03-2018.csv")
income_data = pd.read_csv("datasets/kaggle_income.csv", encoding = "ISO-8859-1")
rent_data = pd.read_csv("datasets/price.csv", encoding = "ISO-8859-1")
states = pd.read_csv("datasets/states.csv")

In [5]:
#Remove null values
#gun_data.dropna(inplace = True)
#income_data.dropna(inplace = True)
gun_data_abbrev = gun_data[['state', 'city_or_county','n_killed', 'n_injured']]

## Data Description

The datasets we are using are US State Population, Gun Violence, US Household Income. Details include state, county, state population, household income, its mean, its median, and its standard deviation. We will be using these sets to find out if the frequency of gun violence has a direct correlation with household income with the effect of population density.
These datasets can be obtained on:
US State Population - https://www.kaggle.com/lucasvictor/us-state-populations-2018?fbclid=IwAR2QTVJfUDl20U0YXWy804-poFuxP1f5IDnJecWh0AMGgoJImjTS0DVNsos
Gun Violence - 
https://www.kaggle.com/jameslko/gun-violence-data
US Household Income -
https://www.kaggle.com/goldenoakresearch/us-household-income-stats-geo-locations


In [ ]:
#gun_data[(gun_data["city_or_county"]=="Mckeesport") & (gun_data["state"]=="Pennsylvania")]

# # clean/merge datasets by city

In [6]:
#TODO: match rows/merge by City
#loop through income data and aggregate a dataframe by city

city_metrics = pd.DataFrame()
count = 0

for index, income_row in income_data.iterrows():
    city = income_row['City']
    state = income_row['State_Name']
    code = income_row['State_ab']
   # print(city)
    #print(state)
    
    #get dataframe of incident rows with matching state and city
    incidents = gun_data_abbrev.loc[(gun_data['state']  == state) & (gun_data['city_or_county'] == city)]
    #print(incidents)
    #compute each city's total incidents and total deaths/ whatever metrics for violence we want to use
    num_incidents = incidents.shape[0]
    total_killed = incidents['n_killed'].sum()
    total_injured = incidents['n_injured'].sum()

    #extract income metrics for city
    mean_inc = income_row['Mean']
    median_inc = income_row['Median']
    stdev_inc = income_row['Stdev']
    
    #construct row of the values we want to describe the city
    city_row = pd.DataFrame(columns = ['state','city','code','mean_inc','median_inc','stdev_inc','num_incidents','total_killed','total_injured'])
    city_row.loc[0] = [state,city,code,mean_inc,median_inc,stdev_inc,num_incidents,total_killed,total_injured]
    #print(city_row)
    
    city_metrics = city_metrics.append(city_row)

In [7]:
#subset rent data to take only large city (avoid duplicates)

city_metrics_rent = pd.DataFrame()
count = 0

for index, rent_row in rent_data.iterrows():
    city = rent_row['City']
    state_abbrev = rent_row['State']
    
    #convert to full state name by comparing to states list
    state = states.loc[states['Abbreviation'] == state_abbrev]['State'].values[0]
    
    print(city)
    print(state)
    
    #get dataframe of incident rows with matching state and city
    incidents = gun_data_abbrev.loc[(gun_data_abbrev['state']  == state) & (gun_data_abbrev['city_or_county'] == city)]
    
    #compute each city's total incidents and total deaths/ whatever metrics for violence we want to use
    num_incidents = incidents.shape[0]
    total_killed = incidents['n_killed'].sum()
    total_injured = incidents['n_injured'].sum()

    #extract rent metrics for city
    latest_rent = rent_row['January 2017']
    
    #construct row of the values we want to describe the city
    city_row = pd.DataFrame(columns = ['state','city', 'latest_rent','num_incidents','total_killed','total_injured'])
    city_row.loc[0] = [state,city,latest_rent,num_incidents,total_killed,total_injured]
    city_metrics_rent = city_metrics_rent.append(city_row)

New York
New York
Los Angeles
California
Chicago
Illinois
Houston
Texas
Philadelphia
Pennsylvania
Phoenix
Arizona
Las Vegas
Nevada
San Antonio
Texas
San Diego
California
Dallas
Texas
San Jose
California
Jacksonville
Florida
San Francisco
California
Indianapolis
Indiana
Austin
Texas
Fort Worth
Texas
Detroit
Michigan
Columbus
Ohio
Memphis
Tennessee
Charlotte
North Carolina
El Paso
Texas
Boston
Massachusetts
Seattle
Washington
Baltimore
Maryland
Denver
Colorado
Washington
District of Columbia
Nashville
Tennessee
Milwaukee
Wisconsin
Tucson
Arizona
Portland
Oregon
Oklahoma City
Oklahoma
Omaha
Nebraska
Albuquerque
New Mexico
Fresno
California
Sacramento
California
Mesa
Arizona
Long Beach
California
Kansas City
Missouri
Virginia Beach
Virginia
Colorado Springs
Colorado
Atlanta
Georgia
Miami
Florida
Oakland
California
Tulsa
Oklahoma
Cleveland
Ohio
Honolulu
Hawaii
Minneapolis
Minnesota
Baton Rouge
Louisiana
New Orleans
Louisiana
Arlington
Texas
Raleigh
North Carolina
Wichita
Kansas
Tampa
Florid

Citrus Heights
California
Buena Park
California
Town of Poughkeepsie
New York
Gary
Indiana
Apopka
Florida
Troy
Michigan
Germantown
Maryland
Richmond
California
Hawthorne
California
San Tan Valley
Arizona
Johnson City
Tennessee
Chino
California
Parma
Ohio
Gulfport
Mississippi
Suffolk
Virginia
Macomb Township
Michigan
Spokane Valley
Washington
Lexington
South Carolina
Farmington Hills
Michigan
Ogden
Utah
Alhambra
California
Leesburg
Virginia
Kent
Washington
Acworth
Georgia
Loveland
Colorado
Sterling
Virginia
Cleveland
Tennessee
Douglasville
Georgia
Redwood City
California
Quincy
Massachusetts
Danbury
Connecticut
Arden-Arcade
California
Parker
Colorado
Hattiesburg
Mississippi
Utica
Michigan
Glen Burnie
Maryland
Lees Summit
Missouri
Dalton
Georgia
Woodstock
Georgia
Ashburn
Virginia
Jackson
Tennessee
Watsonville
California
Rio Rancho
New Mexico
Owensboro
Kentucky
Roswell
Georgia
Lakewood
California
Longmont
Colorado
Newnan
Georgia
Medford
Oregon
Fort Smith
Arkansas
Cranston
Rhode Island
Cam

Lagrange
Georgia
Mililani
Hawaii
Morristown
Tennessee
Upper Darby Township
Pennsylvania
Chillicothe
Ohio
Harlingen
Texas
Grand Island
Nebraska
New Iberia
Louisiana
Novi
Michigan
Blacksburg
Virginia
Klamath Falls
Oregon
Town of Newburgh
New York
Florissant
Missouri
South Whittier
California
Prescott
Arizona
Mishawaka
Indiana
Aloha
Oregon
Riverdale
Georgia
New Brunswick
New Jersey
Saint Clair Shores
Michigan
Parsippany-Troy Hills Township
New Jersey
Tinley Park
Illinois
Kailua
Hawaii
Issaquah
Washington
Castle Rock
Colorado
Easley
South Carolina
Maricopa
Arizona
Yucaipa
California
Marrero
Louisiana
West New York
New Jersey
Glendora
California
Peabody
Massachusetts
Shoreline
Washington
Hacienda Heights
California
Gloucester
New Jersey
Moore
Oklahoma
Waukesha
Wisconsin
Placentia
California
Goshen
Indiana
Eagle Pass
Texas
Euless
Texas
Seminole
Florida
Petersburg
Virginia
San Pablo
California
Plainfield
New Jersey
Troy
New York
Poway
California
Bonita Springs
Florida
South Jordan
Utah
Urbana

Kannapolis
North Carolina
Texarkana
Arkansas
Shelton
Connecticut
Essex
Maryland
Reynoldsburg
Ohio
Brighton
Colorado
Ewing
New Jersey
Roy
Utah
Little Elm
Texas
Oro Valley
Arizona
Waianae
Hawaii
Beverly
Massachusetts
Chelsea
Massachusetts
Kinston
North Carolina
Auburn
Alabama
Hagerstown
Maryland
Wildwood
Missouri
Altadena
California
Bluffton
South Carolina
Saratoga Springs
New York
Laramie
Wyoming
Hilton Head Island
South Carolina
Fond du Lac
Wisconsin
Greeneville
Tennessee
Richfield
Minnesota
Pearl City
Hawaii
Park Ridge
Illinois
Oakville
Missouri
Woburn
Massachusetts
Mt Laurel Township
New Jersey
Anthem
Arizona
Montclair
California
Raeford
North Carolina
Evesham
New Jersey
Rubidoux
California
Dublin
California
Fort Lee
New Jersey
Waycross
Georgia
Covington
Kentucky
Duncanville
Texas
Southaven
Mississippi
Arcadia
Florida
Buffalo Grove
Illinois
Port Angeles
Washington
Trumbull
Connecticut
Wheeling
Illinois
Covington
Louisiana
Boardman
Ohio
Pacifica
California
Beavercreek
Ohio
University 

Lancaster
New York
Villa Rica
Georgia
Catalina Foothills
Arizona
Fort Dodge
Iowa
Rock Springs
Wyoming
Mehlville
Missouri
Emporia
Kansas
Marysville
Ohio
Town of Andover
Massachusetts
Rahway
New Jersey
Auburn
New York
Moncks Corner
South Carolina
Orangevale
California
Rockwall
Texas
Kingsville
Texas
Orange
New Jersey
Fridley
Minnesota
Ponte Vedra Beach
Florida
Plum
Pennsylvania
Rolla
Missouri
Hopewell
Virginia
Stoughton
Massachusetts
Lufkin
Texas
Cabot
Arkansas
Logansport
Indiana
Monterey
California
Holiday
Florida
Town of Rotterdam
New York
Dracut
Massachusetts
Florin
California
Saugus
Massachusetts
Englewood
New Jersey
Newberg
Oregon
Hanover Park
Illinois
Burbank
Illinois
Live Oak
Florida
Tooele
Utah
Marysville
California
Fleming Island
Florida
Plainview
Texas
West Fargo
North Dakota
Eustis
Florida
Glasgow
Kentucky
Fremont
Ohio
Norco
California
Derby
Kansas
Socorro
Texas
Middletown
New Jersey
Deptford Township
New Jersey
Fairhope
Alabama
Bethel Park
Pennsylvania
Shelbyville
Tennessee
O

...

Nashotah
Wisconsin
Spring Park
Minnesota
East Hanover
Pennsylvania
Stillman Valley
Illinois
Three Rivers
Massachusetts
Dewey-Humboldt
Arizona
Cedar Point
North Carolina
Cunningham
Tennessee
Heidelberg
Pennsylvania
Sabin
Minnesota
Zionville
North Carolina
Weissport East
Pennsylvania
Euharlee
Georgia
Keewatin
Minnesota
Velda Village
Missouri
Frazer Township
Pennsylvania
Onarga
Illinois
Clintonville
Wisconsin
Royalston
Massachusetts
Jonesboro
Louisiana
Andale
Kansas
Albany
Illinois
Manitou Beach
Michigan
Mooreland
Indiana
Ferndale
California
Elsie
Michigan
Jackson
New Hampshire
Morrisonville
New York
Hazelhurst
Wisconsin
Upper Falls
West Virginia
Hoisington
Kansas
Elizabeth
Illinois
Eaton
Colorado
Sugar Grove
Virginia
Chaplin
Connecticut
Thomasboro
Illinois
Fletcher
Ohio
Penn Township
Pennsylvania
Cushing
Minnesota
Palmyra
Michigan
Forest Heights
Maryland
Fairplay
Maryland
Freeport
Michigan
Fairview
Michigan
Bridport
Vermont
Geronimo
Oklahoma
Oakfield
Tennessee
Brimfield
Illinois
Grand Co

Blanchard
Idaho
Wyoming
Delaware
Metamora
Illinois
Erie
Illinois
Tecumseh
Kansas
Vergas
Minnesota
Richmond
New Hampshire
Otterbein
Indiana
Hamilton
Montana
Santa Fe
Tennessee
New Castle Northwest
Pennsylvania
Correctionville
Iowa
Chadds Ford Township
Pennsylvania
South Haven
Minnesota
Seabrook
South Carolina
Trenton
Texas
Hot Springs
North Carolina
Effingham
Kansas
Waterford
Virginia
Argo
Alabama
Fountain
Michigan
Olin
North Carolina
Ashton
Maryland
Greenville
California
Town of Wallingford
Vermont
Tallassee
Tennessee
Saint Maries
Idaho
Rapid City
Michigan
Cotopaxi
Colorado
Bowling Green
Virginia
Penn Laird
Virginia
Williamstown
Pennsylvania
Surf City
New Jersey
Tonopah
Nevada
Magnolia
North Carolina
New Hill
North Carolina
Spring Ridge
Pennsylvania
Oxford Township
Pennsylvania
Clay Center
Kansas
Oakley
Michigan
Washington
Louisiana
Shafer
Minnesota
Saint Regis Park
Kentucky
Woodlawn
Tennessee
Cassatt
South Carolina
Sunset
South Carolina
Waterman
Illinois
Coulterville
California
Barnst

Jamestown
North Carolina
Mont Alto
Pennsylvania
Nashwauk
Minnesota
Richmond
Minnesota
Almedia
Pennsylvania
New Holland
Ohio
Catheys Valley
California
Stanchfield
Minnesota
Maricopa
California
Chelsea
Oklahoma
Spruce Pine
Alabama
Aubrey
Texas
Excelsior
Minnesota
Bumpus Mills
Tennessee
Evansville
Wyoming
Haverhill
Florida
Clover
Virginia
Cape Carteret
North Carolina
Wartrace
Tennessee
Flag Pond
Tennessee
McDermott
Ohio
Bluemont
Virginia
Rainier
Oregon
Milan
New Hampshire
Painted Post
New York
Greenfield
Illinois
Primm Springs
Tennessee
Yorkville
Ohio
Raymond
Ohio
Swoope
Virginia
Baldwin
Florida
Baldwin
Georgia
Gray Summit
Missouri
Sherwood
Michigan
Wingina
Virginia
Baidland
Pennsylvania
Combine
Texas
Gosnell
Arkansas
Tillson
New York
Seminole
Alabama
Homeworth
Ohio
Warwick
Maryland
Goode
Virginia
Irvington
Kentucky
Goodland
Indiana
Humansville
Missouri
Winthrop
Iowa
Walnut Grove
California
Fischer
Texas
Hidden Meadows
California
Woodland Beach
Michigan
Bovina
Wisconsin
Walling
Tennessee


Norwich
Ohio
Livingston Manor
New York
Brinnon
Washington
South Bloomfield
Ohio
Anna Maria
Florida
Rhoadesville
Virginia
Rock Creek
Alabama
Star Valley
Arizona
Carroll
New Hampshire
Gloucester Point
Virginia
Newfoundland
New Jersey
Rapidan
Virginia
Iron
Minnesota
Pellston
Michigan
Cologne
Minnesota
Colebrook
Connecticut
Harpswell
Maine
Williston
Tennessee
San Antonio
Florida
Christopher
Illinois
Bivalve
Maryland
Encantada-Ranchito El Calaboz
Texas
Fort Loramie
Ohio
Hillsboro
Tennessee
Highland
Wisconsin
Morrisville
Vermont
Milltown
Wisconsin
Oronoco
Minnesota
Hart'S Location
New Hampshire
Louisburg
North Carolina
Pierz
Minnesota
Downs
Illinois
Cazadero
California
Morrow
Ohio
Kiel
Wisconsin
Munds Park
Arizona
Osage
Minnesota
Arcadia
Michigan
Toquerville
Utah
Kirbyville
Missouri
Telluride
Colorado
Rossville
Georgia
South Roxana
Illinois
Boston
Kentucky
Roseland
Virginia
Airway Heights
Washington
Longbranch
Washington
Peachland
North Carolina
Stone Ridge
New York
Long Creek
Illinois
Towns

Loon Lake
Washington
Ransom Canyon
Texas
Hillsdale
Illinois
Cecilia
Kentucky
Ranchitos Las Lomas
Texas
Borden
Indiana
Town of Coeymans
New York
Onondaga
Michigan
Sharon
Tennessee
Flovilla
Georgia
Pingree Grove
Illinois
China
Texas
Whitethorn
California
Hanover
Illinois
Venus
Texas
Patrick Springs
Virginia
Cabin John
Maryland
London Grove Township
Pennsylvania
Milo
Iowa
Maybrook
New York
Fisher
Illinois
El Paso
Arkansas
Midway
Utah
Bar Nunn
Wyoming
Tullahassee
Oklahoma
Rutledge
Georgia
Redbird
Oklahoma
Napier Township
Pennsylvania
Greenwich Township
New Jersey
Flanagan
Illinois
Georgetown
Colorado
Sperry
Oklahoma
Gheens
Louisiana
Woodlawn Park
Kentucky
Put in Bay
Ohio
Arena
Wisconsin
Darrington
Washington
Dayton
New York
Fort Defiance
Virginia
Free Union
Virginia
Horse Shoe
North Carolina
Kapaau
Hawaii
Meadowlakes
Texas
Seligman
Arizona
Tyaskin
Maryland
Winfield
West Virginia
Big Rock
Tennessee
Jenkinsburg
Georgia
Eagle
Colorado
Mayville
Wisconsin
Sardinia
New York
Sesser
Illinois
Armuc

In [7]:
print(city_metrics)

          state            city mean_inc median_inc stdev_inc num_incidents  \
0       Alabama       Chickasaw    38773      30506     33101             1   
0       Alabama      Louisville    37725      19528     43789             2   
0       Alabama      Columbiana    54606      31930     57348             4   
0       Alabama         Satsuma    63919      52814     47707             0   
0       Alabama  Dauphin Island    77948      67225     54270             0   
0       Alabama         Cullman    50715      42643     35886            15   
0       Alabama    East Brewton    33737      23610     28256             0   
0       Alabama         Coosada    46319      40242     38941             1   
0       Alabama             Eva    57994      39591     47235             0   
0       Alabama       Sylacauga    54807      41712     51359             9   
0       Alabama      Rainsville    47929      38947     44430             1   
0       Alabama         Gadsden    42251      28811 

In [9]:
gun_data_abbrev.loc[(gun_data_abbrev['state'] == 'Alabama') & (gun_data_abbrev['city_or_county'])]

,state,city_or_county,n_killed,n_injured
127,Alabama,Florence,0,4
272,Alabama,Montgomery,3,5
296,Alabama,Huntsville,1,0
315,Alabama,Tuscaloosa,0,3
320,Alabama,Crenshaw,1,0
383,Alabama,Gadsden,0,1
394,Alabama,Huntsville,0,0
396,Alabama,Huntsville,0,0
400,Alabama,Birmingham,1,0
465,Alabama,Huntsville,0,2


## Data Visualization

In [8]:
#TO-DO: Fix typeerror for seaborn plot

city_metrics.plot.scatter(x = 'mean_inc', y = 'num_incidents')

#inc_incidents_scat =  sns.lmplot(x ="mean_inc", y ="num_incidents", data =city_metrics, aspect = 1.5)

ValueError: scatter requires x column to be numeric

In [ ]:
#heatmap of US using plotly

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

us_data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = city_metrics['code'],
    z = city_metrics['num_incidents'].astype(int),
    locationmode = 'USA-states',
    text = city_metrics['num_incidents'],
    marker = go.choropleth.Marker(line = go.choropleth.marker.Line(color = 'rgb(255,255,255)', width=2)),
    colorbar = go.choropleth.ColorBar(title = "Incidents USA")
    )]

us_layout = go.Layout(
    title = go.layout.Title(text = 'Num Incidents Test'),
    geo = go.layout.Geo(
        scope='usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255,255,255)'
        ),
    )

fig = go.Figure(data = us_data, layout = us_layout)
py.iplot(fig, filename = 'test-choropleth-map')


In [37]:
city_metrics.columns

Index(['state', 'city', 'code', 'mean_inc', 'median_inc', 'stdev_inc',
       'num_incidents', 'total_killed', 'total_injured'],
      dtype='object')

# Correlation Analysis

In [39]:
#extract numeric data from city metrics and cast to int
income_vs_incidents = city_metrics[['mean_inc', 'median_inc', 'stdev_inc', 'num_incidents', 'total_killed', 'total_injured']]
income_vs_incidents = income_vs_incidents.astype('int64')
income_vs_incidents.head()

,mean_inc,median_inc,stdev_inc,num_incidents,total_killed,total_injured
0,38773,30506,33101,1,0,0
0,37725,19528,43789,2,0,3
0,54606,31930,57348,4,0,5
0,63919,52814,47707,0,0,0
0,77948,67225,54270,0,0,0


In [42]:
#compute correlation matrix
income_vs_incidents_corrs = income_vs_incidents.corr()
income_vs_incidents_corrs

,mean_inc,median_inc,stdev_inc,num_incidents,total_killed,total_injured
mean_inc,1.000000,0.541713,0.844316,-0.048590,-0.058784,-0.038566
median_inc,0.541713,1.000000,0.329265,-0.050644,-0.054388,-0.043277
stdev_inc,0.844316,0.329265,1.000000,-0.016017,-0.027067,-0.008635
num_incidents,-0.048590,-0.050644,-0.016017,1.000000,0.952464,0.982219
total_killed,-0.058784,-0.054388,-0.027067,0.952464,1.000000,0.902203
total_injured,-0.038566,-0.043277,-0.008635,0.982219,0.902203,1.000000


In [43]:
city_metrics_rent.columns

Index(['state', 'city', 'latest_rent', 'num_incidents', 'total_killed',
       'total_injured'],
      dtype='object')

In [45]:
rent_vs_incidents = city_metrics_rent[['latest_rent','num_incidents','total_killed','total_injured']]
rent_vs_incidents.head()


,latest_rent,num_incidents,total_killed,total_injured
0,2322,17,0,11
0,2753,1066,636,553
0,1632,10814,2104,10427
0,1430,2480,1144,1245
0,1212,2929,890,2318


## Ethics and Privacy
